In [9]:
import numpy as np
import struct
import cv2

In [10]:
def swConv(Kx, Ky, Sx, Sy, mode, relu_en, feat_in, W, bias, feat_out):
    """
    软件卷积操作，参数与hwConv保持一致

    参数:
    - Kx, Ky: 卷积核尺寸
    - Sx, Sy: 竖直和水平方向上的步长
    - mode: 是否需要padding，1为需要，0为不需要
    - relu_en: 是否使用ReLU激活函数
    - feat_in: 输入特征图，形状为 (H_in, W_in, C_in)
    - W: 卷积核权重，形状为 (Kx, Ky, C_in, C_out)
    - bias: 偏置，形状为 (C_out,)
    - feat_out: 输出特征图，形状为 (H_out, W_out, C_out)，预先分配好的数组

    返回:
    - 更新后的feat_out
    """
    H_in, W_in, C_in = feat_in.shape
    _, _, _, C_out = W.shape

    if mode == 1:  # 如果需要padding
        pad_h = (Ky - 1) // 2
        pad_w = (Kx - 1) // 2
        feat_in_padded = np.pad(feat_in, ((pad_h, pad_h), (pad_w, pad_w), (0, 0)), mode='constant', constant_values=0)
    else:
        feat_in_padded = feat_in

    H_out, W_out, _ = feat_out.shape

    for n in range(C_out):  # 对于每个输出通道
        for h in range(H_out):
            for w in range(W_out):
                sum = 0.0
                for c in range(C_in):  # 对于每个输入通道
                    for ky in range(Ky):
                        for kx in range(Kx):
                            sum += W[kx, ky, c, n] * feat_in_padded[h*Sx+kx, w*Sy+ky, c]
                sum += bias[n]
                if relu_en:
                    sum = max(0, sum)
                feat_out[h, w, n] = sum

    return feat_out

In [11]:
def swPool(Kx, Ky, mode, feat_in, feat_out):
    """
    软件池化操作

    参数:
    - Kx, Ky: 池化核尺寸
    - mode: 池化类型（0:MEAN, 1:MIN, 2:MAX）
    - feat_in: 输入特征图，形状为 (H_in, W_in, C_in)
    - feat_out: 输出特征图，形状为 (H_out, W_out, C_out)，预先分配好的数组

    返回:
    - 更新后的feat_out
    """
    H_in, W_in, C_in = feat_in.shape
    H_out, W_out, _ = feat_out.shape

    for c in range(C_in):  # 对于每个通道
        for h in range(H_out):
            for w in range(W_out):
                # 提取当前池化窗口
                window = feat_in[h*Kx:(h+1)*Kx, w*Ky:(w+1)*Ky, c]

                # 根据mode执行不同的池化操作
                if mode == 0:  # MEAN池化
                    feat_out[h, w, c] = np.mean(window)
                elif mode == 1:  # MIN池化
                    feat_out[h, w, c] = np.min(window)
                elif mode == 2:  # MAX池化
                    feat_out[h, w, c] = np.max(window)

    return feat_out

In [12]:
def readbinfile(filename, size):
    f = open(filename, "rb")
    z = []
    for j in range(size):
        data = f.read(4)
        data_float = struct.unpack("f", data)[0]
        z.append(data_float)
    f.close()
    z = np.array(z, dtype = np.float32)
    return z

In [13]:
##################################################
# Conv1
IN_CH1     = 1
IN_WIDTH1  = 28
IN_HEIGHT1 = 28

KERNEL_W1  = 3
STRIDE1    = 1
RELU_EN1   = 1

PADDING1   = int((KERNEL_W1 - 1)/2)

OUT_CH1     = 16
OUT_WIDTH1  = int((IN_WIDTH1  + 2*PADDING1 - KERNEL_W1)/STRIDE1 + 1) # 28
OUT_HEIGHT1 = int((IN_HEIGHT1 + 2*PADDING1 - KERNEL_W1)/STRIDE1 + 1) # 28

##################################################
# Pool1
MODE11      = 2  #mode: 0:MEAN, 1:MIN, 2:MAX
IN_CH11     = OUT_CH1      # 16
IN_WIDTH11  = OUT_WIDTH1   # 28
IN_HEIGHT11 = OUT_HEIGHT1  # 28

KERNEL_W11  = 2

OUT_CH11     = IN_CH11                       # 16
OUT_WIDTH11  = int(IN_WIDTH11 /KERNEL_W11)   # 14
OUT_HEIGHT11 = int(IN_HEIGHT11/KERNEL_W11)   # 14

##################################################
# Conv2
IN_CH2     = OUT_CH11          # 16
IN_WIDTH2  = OUT_WIDTH11       # 14
IN_HEIGHT2 = OUT_HEIGHT11      # 14

KERNEL_W2  = 3
STRIDE2    = 1
RELU_EN2   = 1

PADDING2 = int((KERNEL_W2 - 1)/2)

OUT_CH2     = 32
OUT_WIDTH2  = int((IN_WIDTH2  + 2*PADDING2 - KERNEL_W2)/STRIDE2 + 1) # 14
OUT_HEIGHT2 = int((IN_HEIGHT2 + 2*PADDING2 - KERNEL_W2)/STRIDE2 + 1) # 14

##################################################
# Pool2
MODE21      = 2  #mode: 0:MEAN, 1:MIN, 2:MAX
IN_CH21     = OUT_CH2       # 32
IN_WIDTH21  = OUT_WIDTH2    # 14
IN_HEIGHT21 = OUT_HEIGHT2   # 14

KERNEL_W21  = 2

OUT_CH21     = IN_CH21                      # 32
OUT_WIDTH21  = int(IN_WIDTH21 /KERNEL_W21)  # 7
OUT_HEIGHT21 = int(IN_HEIGHT21/KERNEL_W21)  # 7

##################################################
# FC1
IN_CH3     = OUT_CH21      # 32
IN_WIDTH3  = OUT_WIDTH21   # 7
IN_HEIGHT3 = OUT_HEIGHT21  # 7

KERNEL_W3  = 7
STRIDE3    = 1
RELU_EN3   = 1

OUT_CH3     = 128
OUT_WIDTH3  = int((IN_WIDTH3  - KERNEL_W3)/STRIDE3 + 1) # 1
OUT_HEIGHT3 = int((IN_HEIGHT3 - KERNEL_W3)/STRIDE3 + 1) # 1

##################################################
# FC2
IN_CH4     = OUT_CH3     # 128
IN_WIDTH4  = OUT_WIDTH3  # 1
IN_HEIGHT4 = OUT_HEIGHT3 # 1

KERNEL_W4  = 1
STRIDE4    = 1
RELU_EN4   = 1

OUT_CH4     = 10
OUT_WIDTH4  = int((IN_WIDTH4  - KERNEL_W4)/STRIDE4 + 1) # 1
OUT_HEIGHT4 = int((IN_HEIGHT4 - KERNEL_W4)/STRIDE4 + 1) # 1

In [14]:
!pwd

/content


In [15]:
print("Conv1:\tloading weight... ", end = "")

w_conv1 = readbinfile("./data/W_conv1.bin", KERNEL_W1*KERNEL_W1*IN_CH1*OUT_CH1)
w_conv1 = w_conv1.reshape((KERNEL_W1, KERNEL_W1, IN_CH1, OUT_CH1))

print("done")
print("\tloading bias... ", end = "")

B_conv1 = readbinfile("./data/b_conv1.bin", OUT_CH1)

print("done")
print("Conv2:\tloading weight... ", end = "")

w_conv2 = readbinfile("./data/W_conv2.bin", KERNEL_W2*KERNEL_W2*IN_CH2*OUT_CH2)
w_conv2 = w_conv2.reshape((KERNEL_W2, KERNEL_W2, IN_CH2, OUT_CH2))

print("done")
print("\tloading bias... ", end = "")

B_conv2 = readbinfile("./data/b_conv2.bin",OUT_CH2)

print("done")
print("FC1:\tloading weight... ", end = "")

w_fc1 = readbinfile("./data/W_fc1.bin", KERNEL_W3*KERNEL_W3*IN_CH3*OUT_CH3)
w_fc1 = w_fc1.reshape((KERNEL_W3, KERNEL_W3, IN_CH3, OUT_CH3))

print("done")
print("\tloading bias... ", end = "")

B_fc1 = readbinfile("./data/b_fc1.bin", OUT_CH3)

print("done")
print("FC2:\tloading weight... ", end = "")

w_fc2 = readbinfile("./data/W_fc2.bin", KERNEL_W4*KERNEL_W4*IN_CH4*OUT_CH4)
w_fc2 = w_fc2.reshape((KERNEL_W4, KERNEL_W4, IN_CH4, OUT_CH4))

print("done")
print("\tloading bias... ", end = "")

B_fc2 = readbinfile("./data/b_fc2.bin",OUT_CH4)

print("done")
print("CNN loaded successfully!")

Conv1:	loading weight... done
	loading bias... done
Conv2:	loading weight... done
	loading bias... done
FC1:	loading weight... done
	loading bias... done
FC2:	loading weight... done
	loading bias... done
CNN loaded successfully!


In [16]:
print(w_conv1)

[[[[ 0.18915518 -0.05914321 -0.18613473 -0.11101415 -0.11955081
    -0.00356741 -0.15824226  0.15932235  0.23970585 -0.07581653
    -0.29937842 -0.18055175 -0.3159892   0.2332831  -0.12591499
    -0.09602536]]

  [[ 0.094142    0.06864493 -0.2752557  -0.18756048  0.02991508
    -0.00133922 -0.13442412  0.06690065  0.09845835 -0.03331454
    -0.19297878 -0.18858841 -0.09653205  0.0749345  -0.24786773
     0.15691967]]

  [[-0.09195845  0.09014636 -0.2739124  -0.16061056 -0.14684337
     0.06970645 -0.07392991 -0.17784087  0.2616701   0.04745818
    -0.02385956  0.15177137 -0.0334344  -0.210054   -0.11398687
     0.07393035]]]


 [[[ 0.36136153 -0.13279867 -0.14222036  0.08948099  0.00696284
     0.35737097 -0.02576804 -0.04583993  0.02231593  0.33945912
     0.01133109 -0.315835   -0.02276076 -0.07855142 -0.20654586
     0.01604537]]

  [[ 0.19845167  0.03469291  0.11740304  0.03973265 -0.23030527
     0.369496   -0.04287738 -0.21940912  0.09220141  0.27232662
     0.10045265  0.0572169

In [17]:
import torch

def load_weights(weight_file, shape):
    # 使用numpy读取二进制文件
    weights = np.fromfile(weight_file, dtype=np.float32)

    # 确保读取的权重数量与期望的形状匹配
    weights = weights.reshape(shape)

    # 将numpy数组转换为torch张量
    weights_tensor = torch.from_numpy(weights)
    return weights

# 假设的权重文件路径和权重形状
weight_file = './data/W_conv1.bin'
weight_shape = (3, 3, 1, 16) # 例如：32个输出通道，1个输入通道，5x5的卷积核

# 加载权重
conv1_weights = load_weights(weight_file, weight_shape)

print(conv1_weights)

[[[[ 0.18915518 -0.05914321 -0.18613473 -0.11101415 -0.11955081
    -0.00356741 -0.15824226  0.15932235  0.23970585 -0.07581653
    -0.29937842 -0.18055175 -0.3159892   0.2332831  -0.12591499
    -0.09602536]]

  [[ 0.094142    0.06864493 -0.2752557  -0.18756048  0.02991508
    -0.00133922 -0.13442412  0.06690065  0.09845835 -0.03331454
    -0.19297878 -0.18858841 -0.09653205  0.0749345  -0.24786773
     0.15691967]]

  [[-0.09195845  0.09014636 -0.2739124  -0.16061056 -0.14684337
     0.06970645 -0.07392991 -0.17784087  0.2616701   0.04745818
    -0.02385956  0.15177137 -0.0334344  -0.210054   -0.11398687
     0.07393035]]]


 [[[ 0.36136153 -0.13279867 -0.14222036  0.08948099  0.00696284
     0.35737097 -0.02576804 -0.04583993  0.02231593  0.33945912
     0.01133109 -0.315835   -0.02276076 -0.07855142 -0.20654586
     0.01604537]]

  [[ 0.19845167  0.03469291  0.11740304  0.03973265 -0.23030527
     0.369496   -0.04287738 -0.21940912  0.09220141  0.27232662
     0.10045265  0.0572169

In [18]:
h_conv1 = np.zeros(shape=(OUT_HEIGHT1, OUT_WIDTH1, OUT_CH1), dtype=np.float32)
h_pool1 = np.zeros(shape=(OUT_HEIGHT11, OUT_WIDTH11, OUT_CH11), dtype = np.float32)
h_conv2 = np.zeros(shape=(OUT_HEIGHT2, OUT_WIDTH2, OUT_CH2), dtype = np.float32)
h_pool2 = np.zeros(shape=(OUT_HEIGHT21, OUT_WIDTH21, OUT_CH21), dtype = np.float32)
h_fc1 = np.zeros(shape=(OUT_HEIGHT3, OUT_WIDTH3, OUT_CH3), dtype = np.float32)
h_fc2 = np.zeros(shape=(OUT_HEIGHT4, OUT_WIDTH4, OUT_CH4), dtype = np.float32)
image = np.zeros(shape=(IN_HEIGHT1, IN_WIDTH1, IN_CH1), dtype = np.float32)


In [19]:
image1 = cv2.imread("./data/2.jpg", cv2.IMREAD_GRAYSCALE).astype(np.float32)
for r in range(IN_HEIGHT1):
    for c in range(IN_WIDTH1):
        for ch in range(IN_CH1):
            image[r][c][ch] = (255 - image1[r][c])/255

print("Finish reading image.")

# Conv1
swConv(KERNEL_W1, KERNEL_W1, STRIDE1, STRIDE1, 1, RELU_EN1, image, w_conv1, B_conv1, h_conv1)
swPool(KERNEL_W11, KERNEL_W11, MODE11, h_conv1, h_pool1)
# Conv2
swConv(KERNEL_W2, KERNEL_W2, STRIDE2, STRIDE2, 1, RELU_EN2, h_pool1, w_conv2, B_conv2, h_conv2)
swPool(KERNEL_W21, KERNEL_W21, MODE21, h_conv2, h_pool2)
# FC1
swConv(KERNEL_W3, KERNEL_W3, STRIDE3, STRIDE3, 0, RELU_EN3, h_pool2, w_fc1, B_fc1, h_fc1)
# FC2
swConv(KERNEL_W4, KERNEL_W4, STRIDE4, STRIDE4, 0, RELU_EN4, h_fc1, w_fc2, B_fc2, h_fc2)

MAX = h_fc2[0][0][0]
result = 0
for ch in range(1, OUT_CH4):
    if (h_fc2[0][0][ch] > MAX):
        MAX = h_fc2[0][0][ch]
        result = ch

print("The image was recognized as " + str(result))

Finish reading image.
The image was recognized as 2
